# Installations & Configuration

In [ ]:
!pip install kaggle

import os
import torch
import pandas as pd
import torch.nn as nn
import torchvision.transforms as transforms

from PIL import Image
from google.colab import files
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')
kaggle_json_path = '/content/drive/MyDrive/ColabNotebooks/A5/kaggle.json'

# Copy kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp {kaggle_json_path} ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


# Data Setup

In [ ]:
# Get the movie posters from kaggle
!kaggle datasets download -d rezaunderfit/48k-imdb-movies-with-posters > /dev/null 2>&1
!unzip -q 48k-imdb-movies-with-posters.zip

# Load title basics
tsv_path = '/content/drive/MyDrive/ColabNotebooks/A5/title.basics.tsv'
title_basics = pd.read_csv(tsv_path, sep='\t', na_values='\\N')

# List all files in the Poster directory
poster_dir = 'Poster'
poster_files = []
for root, _, files in os.walk(poster_dir):
    for file in files:
        if file.endswith('.jpg'):
            poster_files.append(os.path.join(root, file))

# Extract tconst and startYear from file paths
poster_info = []
for file_path in poster_files:
    parts = file_path.split('/')
    start_year = parts[1]
    tconst = parts[2]
    poster_info.append((start_year, tconst))

# Convert to DataFrame
poster_df = pd.DataFrame(poster_info, columns=['startYear', 'tconst'])

# Ensure startYear is an integer
poster_df['startYear'] = poster_df['startYear'].astype(int)
title_basics['startYear'] = title_basics['startYear'].astype(float).fillna(0).astype(int)  # Handle missing startYear and convert to int

# Merge with title_basics to keep only relevant records
title_basics_filtered = pd.merge(title_basics, poster_df, on=['startYear', 'tconst'])

# Create your data splits
train_metadata, test_metadata = train_test_split(title_basics_filtered, test_size=0.2, random_state=42)
train_metadata, val_metadata = train_test_split(train_metadata, test_size=0.25, random_state=42)
print(f"Train size: {len(train_metadata)}, Validation size: {len(val_metadata)}, Test size: {len(test_metadata)}")



# Define the image transformations
image_transforms = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class MovieDataset(Dataset):
    def __init__(self, metadata, img_dir, transform=None, genre_to_index=None):
        self.metadata = metadata
        self.img_dir = img_dir
        self.transform = transform
        self.genre_to_index = genre_to_index

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        tconst = self.metadata.iloc[idx]['tconst']
        start_year = self.metadata.iloc[idx]['startYear']
        img_name = os.path.join(self.img_dir, str(start_year), tconst, f"{tconst}.jpg")
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        genres = self.metadata.iloc[idx]['genres']
        genre_tensor = self.genres_to_tensor(genres)
        return image, genre_tensor

    def genres_to_tensor(self, genres):
        first_genre = genres.split(',')[0] if pd.notna(genres) else 'Unknown'
        genre_index = self.genre_to_index.get(first_genre, self.genre_to_index['Unknown'])
        return torch.tensor(genre_index, dtype=torch.long)

# Create a mapping from genre to index
all_genres = set(g.split(',')[0] for g in title_basics_filtered['genres'].dropna())
genre_to_index = {genre: idx for idx, genre in enumerate(all_genres)}
genre_to_index['Unknown'] = len(genre_to_index)

# Directory containing images
img_dir = 'Poster'

# Create datasets
train_dataset = MovieDataset(metadata=train_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)
val_dataset = MovieDataset(metadata=val_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)
test_dataset = MovieDataset(metadata=test_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Example of using the DataLoader
for images, genres in data_loader:
    print(images.shape)  # Shape: (batch_size, 3, 182, 268)
    print(genres.shape)  # Shape: (batch_size,)
    break

<ipython-input-2-c0251d199d45>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv(tsv_path, sep='\t', na_values='\\N')
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([32, 3, 182, 268])
torch.Size([32])


# Transfer Learning --> Fine Tuning (Training)

In [ ]:
# Set up model for fine tuning
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.aux_logits = False  # Disable auxiliary logits
num_genres = len(genre_to_index)
model.fc = nn.Linear(model.fc.in_features, num_genres)  # Adjust the final layer

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop with validation
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, genres in train_loader:
        images = images.to(device)
        genres = genres.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, genres)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_images, val_genres in val_loader:
            val_images = val_images.to(device)
            val_genres = val_genres.to(device)
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_genres).item()

    val_loss /= len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss}")

    # Save the model if validation loss decreases
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

print("Training complete.")



(40205, 9)
      tconst titleType                                 primaryTitle  \
0  tt0000001     short                                   Carmencita   
1  tt0000003     short                               Pauvre Pierrot   
2  tt0000005     short                             Blacksmith Scene   
3  tt0000007     short  Corbett and Courtney Before the Kinetograph   
4  tt0000008     short       Edison Kinetoscopic Record of a Sneeze   

                                 originalTitle  isAdult  startYear  endYear  \
0                                   Carmencita      0.0       1894      NaN   
1                               Pauvre Pierrot      0.0       1892      NaN   
2                             Blacksmith Scene      0.0       1893      NaN   
3  Corbett and Courtney Before the Kinetograph      0.0       1894      NaN   
4       Edison Kinetoscopic Record of a Sneeze      0.0       1894      NaN   

  runtimeMinutes                    genres  
0            1.0         Documentary,Short

# Transfer Learning --> Fine Tuning (Testing)

In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test set
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for test_images, test_genres in test_loader:
        test_images = test_images.to(device)
        test_genres = test_genres.to(device)
        test_outputs = model(test_images)
        test_loss += criterion(test_outputs, test_genres).item()
        _, predicted = torch.max(test_outputs, 1)
        total += test_genres.size(0)
        correct += (predicted == test_genres).sum().item()

test_loss /= len(test_loader)
test_accuracy = correct / total
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")